In [ ]:
using Plots, LinearAlgebra, FFTW, DeltaRCWA

# Stage by Stage

**Input Data**

Two layers with thickness d, size L.
Assume Gaussian function delta layer
z = 0 at the top of layer 1, x = 0 on the left of each cell

In [ ]:
L = 1 # width of cell [arb]
λ₀ = 1.2 # incoming light wavelength [units of L]
ω₀ = 2*pi/λ₀

ϵ₁ = 1 # air in the first layer
d₁ = 2 # thickness of first layer [units of L]
ϵ₂ = 1 # air in the second layer
d₂ = 2 # thickness of second layer [units of L]

delta(n) = n == 0
f0(x, z) = @. 1+ 5*delta(x-x) * delta(z-d₁) # constant surface dielectric
f1(x, z) = @. 1+ exp(-(x-L/2)^2/(2*L^2/16)) * delta(z-d₁) # gaussian function in x

nvec = 0:99 # 100 modes

In [ ]:
kₓ = 2*pi*fftfreq(length(nvec), length(nvec)/L) 

In [ ]:
#### initial wave
xvec = range(0, L, length=length(nvec)+1)[1:length(nvec),]

u_p = [mod(n, 3) == 0 ? 1 : 0 for n in nvec] # all even modes set to 1, p is the positive side
u_n = [mod(n, 4) == 0 ? 1 : 0 for n in nvec] # all multiples of 3 set to 1, n is the negative side

E_in_p = ifft(u_p)
E_in_n = ifft(u_n)

plot(xvec, abs.(E_in_p))
plot!(xvec, abs.(E_in_n))



In [ ]:
#### propagate through first layer. (right before boundary)
β₁ = @. sqrt(Complex(ω₀^2*ϵ₁ - kₓ^2)) # propagation constant in first layer
β₂ = @. sqrt(Complex(ω₀^2*ϵ₂ - kₓ^2)) # propagation constant in second layer

u_p = u_p .* exp.(1im*β₁*d₁)
u_n = u_n .* exp.(1im*β₂*d₂)

E_in_p = ifft(u_p)
E_in_n = ifft(u_n)

plot(xvec, abs.(E_in_p))
plot!(xvec, abs.(E_in_n))

In [ ]:
β₁

In [ ]:
#### Scattering matrix (one mode)
k = ω₀
βₙ = @. sqrt(Complex(ω₀^2*ϵ₁ - kₓ^2))


M = Matrix(Diagonal(repeat([1], length(nvec))*1im))
N = Matrix(Diagonal(repeat([6], length(nvec))*1im))

A = [-Diagonal(βₙ)-k*M -Diagonal(βₙ)-k*M; -Diagonal(βₙ)-k*N Diagonal(βₙ)+k*N]
B = [-Diagonal(βₙ)+k*M -Diagonal(βₙ)+k*M; Diagonal(βₙ)+k*N -Diagonal(βₙ)-k*N]

S = A\B


In [ ]:
ip = [findall(isreal, βₙ); findall(isreal, βₙ).+length(βₙ)]
Sp = S[ip,ip]

In [ ]:
norm(Sp'Sp-I)/norm(Sp'Sp)

In [ ]:
norm(S'S-I)/norm(S'S)

In [ ]:
#### Scattering matrix (all modes)
k = ω₀
# M = Matrix(Diagonal(rand(length(nvec))*1im))
# N = Matrix(Diagonal(rand(length(nvec))*1im))
M = Matrix(Diagonal(repeat([1], length(nvec))*1im))
N = Matrix(Diagonal(repeat([6], length(nvec))*1im))
A = [-Diagonal(β₁)-k*ifft(fft(M, 2), 1)    -Diagonal(β₁)-k*ifft(fft(M, 2), 1);
    -Diagonal(β₁)-k*ifft(fft(N, 2), 1)     Diagonal(β₁)+k*ifft(fft(N, 2), 1)]
B = [Diagonal(β₁)+k*ifft(fft(M, 2), 1)    Diagonal(β₁)+k*ifft(fft(M, 2), 1);
    Diagonal(β₁)+k*ifft(fft(N, 2), 1)      -Diagonal(β₁)-k*ifft(fft(N, 2), 1)]

#     A = [-2*Diagonal(β)-k*ifft(fft(M+N, 2), 1)    -k*ifft(fft(M-N, 2), 1);
#         -k*ifft(fft(N-M, 2), 1)                   2*Diagonal(β)+k*ifft(fft(N+M, 2), 1)]
#     B = [-2*Diagonal(β)+k*ifft(fft(M+N, 2), 1)    k*ifft(fft(M-N, 2), 1);
#         k*ifft(fft(N-M, 2), 1)                  2*Diagonal(β)-k*ifft(fft(N+M, 2), 1)]

S = A\B


In [ ]:
norm(S'S-I)/norm(S'S)

In [ ]:
nvec = 0:99
M = Matrix(Diagonal(repeat([2], length(nvec))*1im))
N = Matrix(Diagonal(repeat([0.5], length(nvec))*1im))

# M = Matrix(Diagonal(rand(length(nvec))*10im))
# N = Matrix(Diagonal(rand(length(nvec))*1im))

# M = Matrix(Diagonal([(n > 45 && n < 75) ? 1 : 0 for n in nvec])*10im)
# N = Matrix(Diagonal([(n > 45 && n < 75) ? 1 : 0 for n in nvec])*10im)

u_p = [n == 0 ? 1 : 0 for n in nvec]
u_n = [n == 0 ? 0 : 0 for n in nvec]

# plot(abs.(u_p), label="region 1")
# display(plot!(abs.(u_n), label="region 2"))

λ_vec = 0.25:1e-2:2
N_vec = 0.5:0.1:2
T = zeros(0)
for λi in λ_vec
#     N = Matrix(Diagonal(repeat([Ni], length(nvec))*1im))
    Ti, phasei = get_transmissivity_normalincident(M, N, 1, λi, u_p, u_n)
    append!(T, Ti)
end

In [ ]:
plot(λ_vec, T, ylim=(-0.1, 1.1))

In [ ]:
nvec = 0:2
M = Matrix(Diagonal(repeat([1], length(nvec))*0im))
N = 4*M

u_p = [n == 0 ? 1 : 0 for n in nvec]
u_n = [n == 0 ? 0 : 0 for n in nvec]

S, β = get_all_modes(M, N, 1, 0.5)
u_in = [u_p; u_n]
u_out = S*u_in
display(β)
# display(S)
display(u_in)
display(u_out)

# Debugging

In [ ]:
#### trying to debug (5/6/21)
nvec = 0:3
λ = 0.25
L = 1

Y_vec = 0:0.01:10
T = zeros(0)
for Yi in Y_vec
    Y = Matrix(Diagonal(repeat([1], length(nvec))*Yi*(1im)))
    # Y = Matrix(Diagonal([0; 1; 2; 3; 2; 1; 0])*1im)

    u_p = [n == 0 ? 1 : 0 for n in nvec]
    u_n = [n == 0 ? 0 : 0 for n in nvec]

    ω₀ = 2*pi/λ
    k = ω₀
    nvec = 0:length(Y)^0.5-1 # 100 modes
    kₓ = 2*pi*fftfreq(length(nvec), length(nvec)/L) 
    β = @. sqrt(Complex(ω₀^2 - kₓ^2))
    # display(β)

    A = [Diagonal(β)+0.5*k*ifft(fft(Y, 2), 1)    Diagonal(β)+0.5*k*ifft(fft(Y, 2), 1);
        Diagonal(β)+2*k*ifft(fft(Y, 2), 1)     -Diagonal(β)-2*k*ifft(fft(Y, 2), 1)]
    B = [Diagonal(β)-0.5*k*ifft(fft(Y, 2), 1)    Diagonal(β)-0.5*k*ifft(fft(Y, 2), 1);
        Diagonal(β)-2*k*ifft(fft(Y, 2), 1)      -Diagonal(β)+2*k*ifft(fft(Y, 2), 1)]

    S = A\B
    # display(S)

    u_in = [u_p; u_n]
    u_out = S*u_in

    append!(T, abs(u_out[5]))
#     println(abs(u_out[5])^2 + abs(u_out[1])^2)
end

plot(Y_vec, T)

In [ ]:
print(norm(S'S-I)/norm(S'S))
display(heatmap(abs.(S'S), yflip=true))
display(heatmap(abs.(S), yflip=true))

In [ ]:
nvec = 0:3
M = Matrix(Diagonal(repeat([0.1], length(nvec))*1im))
N = Matrix(Diagonal(repeat([100], length(nvec))*1im))

u_p = [n == 0 ? 1 : 0 for n in nvec]
u_n = [n == 0 ? 0 : 0 for n in nvec]

# S, β = get_all_modes(M, N, 1, 0.5)
L = 1
λ = 0.75
ω₀ = 2*pi/λ
k = ω₀
nvec = 0:length(M)^0.5-1 # 100 modes
kₓ = 2*pi*fftfreq(length(nvec), length(nvec)/L) 
β = @. sqrt(Complex(ω₀^2 - kₓ^2))
# display(β)

A = [-Diagonal(β)-k*ifft(fft(M, 2), 1)    -Diagonal(β)-k*ifft(fft(M, 2), 1);
    -Diagonal(β)-k*ifft(fft(N, 2), 1)     Diagonal(β)+k*ifft(fft(N, 2), 1)]
B = [-Diagonal(β)+k*ifft(fft(M, 2), 1)    -Diagonal(β)+k*ifft(fft(M, 2), 1);
    -Diagonal(β)+k*ifft(fft(N, 2), 1)     Diagonal(β)-k*ifft(fft(N, 2), 1)]
S = A\B
# display(S)

u_in = [u_p; u_n]
u_out = S*u_in
display(u_in)
display(u_out)
println("reflection: ", abs(u_out[1])^2)
println("transmission: ", abs(u_out[5])^2)
println("energy: ", abs(u_out[1])^2 + abs(u_out[5])^2)

In [ ]:
nvec = 0:3
M = Matrix(Diagonal(repeat([2], length(nvec))*1im))
N = Matrix(Diagonal(repeat([2], length(nvec))*1im))

u_p = [n == 0 ? 1 : 0 for n in nvec]
u_n = [n == 0 ? 0 : 0 for n in nvec]

# S, β = get_all_modes(M, N, 1, 0.5)
L = 1
λ = 0.75
ω₀ = 2*pi/λ
k = ω₀
nvec = 0:length(M)^0.5-1 # 100 modes
kₓ = 2*pi*fftfreq(length(nvec), length(nvec)/L) 
β = @. sqrt(Complex(ω₀^2 - kₓ^2))
# display(β)

T = zeros(0)
Nvec = 1:1:100
for Ni in Nvec
    N = Matrix(Diagonal(repeat([Ni], length(nvec))*1im))
    A = [-Diagonal(β)-k*ifft(fft(M, 2), 1)    -Diagonal(β)-k*ifft(fft(M, 2), 1);
        -Diagonal(β)-k*ifft(fft(N, 2), 1)     Diagonal(β)+k*ifft(fft(N, 2), 1)]
    B = [-Diagonal(β)+k*ifft(fft(M, 2), 1)    -Diagonal(β)+k*ifft(fft(M, 2), 1);
        -Diagonal(β)+k*ifft(fft(N, 2), 1)     Diagonal(β)-k*ifft(fft(N, 2), 1)]
    S = A\B
    # display(S)

    u_in = [u_p; u_n]
    u_out = S*u_in
#     display(u_in)
#     display(u_out)
#     println("reflection: ", abs(u_out[1]))
#     println("transmission: ", abs(u_out[5]))
#     println("energy: ", abs(u_out[1])^2 + abs(u_out[5])^2)
    append!(T, abs(u_out[5]))

end
plot(Nvec, T)

In [ ]:
heatmap(abs.(S'S))

In [ ]:
heatmap(abs.(S))

In [ ]:
#### apply S matrix
u_out = S*[uy_p ; uy_n]
uy_p = u_out[1:Int(length(u_out)/2)]
uy_n = u_out[Int(length(u_out)/2)+1:Int(length(u_out))]


In [ ]:
#### right after boundary
E_outp = ifft(uy_p)
E_outn = ifft(uy_n)

# plot(xvec, abs.(E_outp))
# plot!(xvec, abs.(E_outn))

plot(nvec, abs.(uy_p))
plot!(nvec, abs.(uy_n))


In [ ]:
#### check conservation of energy
abs(dot(conj(E_in_p), E_in_p))
abs(dot(conj(E_in_n), E_in_n))

abs(dot(conj(E_outp), E_outp))
abs(dot(conj(E_outn), E_outn))

In [ ]:
#### propagate through second layer
# u_y_p = @. u_y_p * exp(1im * b1 * d1)
# u_y_n = @. u_y_n * exp(-1im * b2 * d2)

println(u_y_p)
println(u_y_n)

E_outp = ifft(u_y_p)
E_outn = ifft(u_y_n)
plot(xvec, abs.(E_outp))
# plot(xvec, abs.(E_outn))

In [ ]:
#### OLD Not Needed Anymore OLD ####

Nx = 10 # number of discretized points in x

h = L/Nx
rows = vcat([[i, i, i] for i=1:Nx]...)
cols = vcat(vcat([[1,2,3]], [[i-1, i, i+1] for i=2:Nx-1], [Nx.-[2,1,0]])...)
vals = repeat([1,-2,1]./h^2, Nx)
A = sparse(rows, cols, vals, Nx, Nx) + w0^2*e1*I

vals, vecs = eigs(Array(A))

print(vals)
# print(vecs)

In [ ]:
#### trying to debug (5/19/21)
nvec = 0:3
ϵ = repeat([1], length(nvec))
μ = repeat([1], length(nvec))

u_p = [n == 0 ? 1 : 0 for n in nvec]
u_n = [n == 0 ? 0 : 0 for n in nvec]

# S, β = get_all_modes(M, N, 1, 0.5)
L = 1
λ = 0.75
ω₀ = 2*pi/λ
k = ω₀
# nvec = 0:length(M)^0.5-1 # 100 modes
kₓ = 2*pi*fftfreq(length(nvec), length(nvec)/L) 
β = @. sqrt(Complex(ω₀^2 - kₓ^2))
# display(β)

M = 1im*(k*Matrix(Diagonal(ϵ)))/2
N = 1im*2/k*Matrix(Diagonal(1 ./μ))

A = [-Diagonal(β)-k*ifft(fft(M, 2), 1)    -Diagonal(β)-k*ifft(fft(M, 2), 1);
    -Diagonal(β)-k*ifft(fft(N, 2), 1)     Diagonal(β)+k*ifft(fft(N, 2), 1)]
B = [-Diagonal(β)+k*ifft(fft(M, 2), 1)    -Diagonal(β)+k*ifft(fft(M, 2), 1);
    -Diagonal(β)+k*ifft(fft(N, 2), 1)     Diagonal(β)-k*ifft(fft(N, 2), 1)]
S = A\B

u_in = [u_p; u_n]
u_out = S*u_in
display(u_in)
display(u_out)
println("reflection: ", abs(u_out[1])^2)
println("transmission: ", abs(u_out[5])^2)
println("energy: ", abs(u_out[1])^2 + abs(u_out[5])^2)

In [ ]:
#### 5/19/21
nvec = 0:99
# ϵ = repeat([1e-9], length(nvec)) ## susceptibility
μ = repeat([1e-9], length(nvec))
ϵ = [(n > 40 && n < 60) ? 1e-9 : 1 for n in nvec]
# μ = [(n > 48 && n < 52) ? 1e-9 : 1 for n in nvec]

u_p = [n == 0 ? 1 : 0 for n in nvec]
u_n = [n == 0 ? 0 : 0 for n in nvec]

# S, β = get_all_modes(M, N, 1, 0.5)
L = 1
λᵥ = 0.1:0.01:2
T = zeros(0)
for λ in λᵥ
    ω₀ = 2*pi/λ
    k = ω₀
    kₓ = 2*pi*fftfreq(length(nvec), length(nvec)/L)
    β = @. sqrt(Complex(ω₀^2 - kₓ^2))
    # display(β)

    M = 1im*(k*Matrix(Diagonal(ϵ)))/2
    N = 1im*2/k*Matrix(Diagonal(1 ./μ))

    A = [-Diagonal(β)-k*ifft(fft(M, 2), 1)    -Diagonal(β)-k*ifft(fft(M, 2), 1);
        -Diagonal(β)-k*ifft(fft(N, 2), 1)     Diagonal(β)+k*ifft(fft(N, 2), 1)]
    B = [-Diagonal(β)+k*ifft(fft(M, 2), 1)    -Diagonal(β)+k*ifft(fft(M, 2), 1);
        -Diagonal(β)+k*ifft(fft(N, 2), 1)     Diagonal(β)-k*ifft(fft(N, 2), 1)]
    S = A\B
    # display(S)

    u_in = [u_p; u_n]
    u_out = S*u_in
    # display(u_in)
    # display(u_out)
#     println("reflection: ", abs(u_out[1])^2)
#     println("transmission: ", abs(u_out[5])^2)
#     println("energy: ", abs(u_out[1])^2 + abs(u_out[5])^2, "\n")
    append!(T, abs(u_out[101])^2)
end
plot(λᵥ, T, xlabel="λ/L", ylabel="transmission")

# Checking with Carlos's Solver

In [ ]:
#### Matching Carlos's solver
k = 10.0;
λ = 2*pi/k;
θ = -π/2.0; # incidence angle (measured with respect to the x axis)
α = k*cos(θ);
β = k*sin(θ);
uInc(x,y)= @. exp(1im*α*x+1im*β*y);  # incident planewave

θᵗ = -π/8;    # transmitted field angle
d  = cos(θᵗ)-cos(θ); 
L = 2*(2*π)/(k*abs(d));  # Unit cell width
M₀(x) = @. -sin(θ)*(1+exp(1im*k*d*x));
N₀(x) = @. -sin(θ)*(1-exp(1im*k*d*x));

nvec = 0:99;
dx = L/length(nvec)
xvec = [n*dx-L/2 for n in nvec]
kₓ = 2*pi*fftfreq(length(nvec), 1/dx)
β = @. sqrt(Complex(k^2 - kₓ^2))

u_p = fft(uInc(xvec, 0))/length(nvec)
u_n = zeros(length(nvec))
M = Matrix(Diagonal(M₀(xvec)))
N = Matrix(Diagonal(N₀(xvec)))
# display(N)


A = [-Diagonal(β)-k*ifft(fft(M, 2), 1)    -Diagonal(β)-k*ifft(fft(M, 2), 1);
    -Diagonal(β)-k*ifft(fft(N, 2), 1)     Diagonal(β)+k*ifft(fft(N, 2), 1)]
B = [-Diagonal(β)+k*ifft(fft(M, 2), 1)    -Diagonal(β)+k*ifft(fft(M, 2), 1);
    -Diagonal(β)+k*ifft(fft(N, 2), 1)     Diagonal(β)-k*ifft(fft(N, 2), 1)]
S = A\B
# display(S)

u_in = [u_p; u_n]
u_out = S*u_in
u_out_p = u_out[1:length(nvec)]
u_out_n = u_out[length(nvec)+1:2*length(nvec)]
display(u_out_p)
display(u_out_n)
plot(xvec, abs2.(ifft(u_out_p)))
plot!(xvec, abs2.(ifft(u_out_n)))

In [ ]:
u_out_p

In [ ]:
xPlot = -L/2:0.001:L/2;

paramPlot = plot(xPlot,real.(M₀.(xPlot)))

paramPlot = plot!(xPlot,imag.(M₀.(xPlot))) 

paramPlot = plot!(xPlot,real.(N₀.(xPlot))) 

paramPlot = plot!(xPlot,imag.(N₀.(xPlot))) 

plot(paramPlot,lw=3,linestyle=[:solid :dash :solid :dash],label = ["Re M" "Im M" "Re N" "Im N"],frame=:box)
xlabel!("x");ylabel!("M, N")

In [ ]:
E1(x,z) = sum([u_out_p[n+1]*exp(-1im*kₓ[n+1]*x)*exp(1im*β[n+1]*z) for n in nvec])+sum([u_p[n+1]*exp(-1im*kₓ[n+1]*x)*exp(1im*β[n+1]*z) for n in nvec])
E2(x,z) = sum([u_out_n[n+1]*exp(-1im*kₓ[n+1]*x)*exp(-1im*β[n+1]*z) for n in nvec])

xview = -L:0.02:L 
zview = 0:0.01:L
zview2 = -L:0.01:0
Emat = zeros(Complex{Float64}, 0)
Emat2 = zeros(Complex{Float64}, 0)
for (z1, z2) in zip(zview, zview2)
    append!(Emat, E1.(xview, z1))
    append!(Emat2, E2.(xview, z2))
end
Emat = reshape(Emat,length(xview),:)
Emat2 = reshape(Emat2,length(xview),:)
# display(Emat)
# display(Emat2)


In [ ]:
plt1 = heatmap(xview, zview2, transpose(real.(Emat2)))
plt1 = heatmap!(xview, zview, transpose(real.(Emat)))
display(plt1)
plt2 = heatmap(xview, zview2, transpose(imag.(Emat2)))
plt2 = heatmap!(xview, zview, transpose(imag.(Emat)))
display(plt2)
plt3 = heatmap(xview, zview2, transpose(abs2.(Emat2)))
plt3 = heatmap!(xview, zview, transpose(abs2.(Emat)))
display(plt3)